In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns


import time
import random
import sys

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df1 = pd.read_csv('../input/netflix-prize-data/combined_data_1.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])

In [ ]:
print(df1.shape)
df1.head(3)

In [ ]:
start_time = time.time()

p = df1.groupby('Cust_Id')['Cust_Id'].agg(['count'])

print("--- Pandas' groupby took %s seconds ---" % (time.time() - start_time))
print("--- Pandas' groupby took %s space ---" % str(p.shape))

In [ ]:
p.head(3)

In [ ]:
class AMS:
    def __init__(self, seed):
        self.X = 0
        self.seed = seed
        
    def add_element(self, item):
        coin = random.choice([1, -1])
        self.X += coin

    def estimate(self):
        return self.X**2

In [ ]:
x=0
for i in range(100):
    x += random.choice([1, -1])
print(x)

In [ ]:
class AMSMean:
    def __init__(self, number_of_copies=10, seed=42):
        self.X = [AMS(i+number_of_copies*seed) for i in range(number_of_copies)]
        
    def add_element(self, item):
        for ams in self.X:
            ams.add_element(item)

    def estimate(self):
        return np.mean(np.array([x.estimate() for x in self.X]))

In [ ]:
class AMSMedian:
    def __init__(self, number_of_copies=10, seed=42):
        self.X = [AMS(i+number_of_copies*seed) for i in range(number_of_copies)]
        
    def add_element(self, item):
        for ams in self.X:
            ams.add_element(item)

    def estimate(self):
        return np.median(np.array([x.estimate() for x in self.X]))

In [ ]:
class AMSFinalMedian:
    def __init__(self, number_of_copies=10, number_of_beta_copies=10):
        seed = random.randint(0,10**8)
        self.X = [AMSMean(number_of_beta_copies, i*seed) for i in range(number_of_copies)]
        
    def add_element(self, item):
        for ams in self.X:
            ams.add_element(item)

    def estimate(self):
        return int(np.median(np.array([x.estimate() for x in self.X])))

In [ ]:
class AMSFinalMean:
    def __init__(self, number_of_copies=10, number_of_beta_copies=10):
        seed = random.randint(0,10**8)
        self.X = [AMSMedian(number_of_beta_copies, i*seed) for i in range(number_of_copies)]
        
    def add_element(self, item):
        for ams in self.X:
            ams.add_element(item)

    def estimate(self):
        return int(np.mean(np.array([x.estimate() for x in self.X])))

In [ ]:
 def run_ams_final_median(number_of_copies=10, number_of_beta_copies=10):
    start_time = time.time()

    ams = AMSFinalMedian(number_of_copies, number_of_beta_copies)

    for index, row in df1[:100000].iterrows():
        ams.add_element(row['Cust_Id'])
#         if index % 10000 == 0:
# #             print(ams.X)
#             print("Saw %d elements" % index)

    print("AMSFinalMedian estimated %d elements" % ams.estimate()) 
    print("AMSFinalMedian took %s seconds" % (time.time() - start_time))
    return ams.estimate()

In [ ]:
 def run_ams_final_mean(number_of_copies=10, number_of_beta_copies=10):
    start_time = time.time()

    ams = AMSFinalMean(number_of_copies, number_of_beta_copies)

    for index, row in df1[:100000].iterrows():
        ams.add_element(row['Cust_Id'])
#         if index % 10000 == 0:
#             print("Saw %d elements" % index)

    print("AMSFinalMean estimated %d elements" % ams.estimate()) 
    print("AMSFinalMean took %s seconds" % (time.time() - start_time))
    return ams.estimate()

In [ ]:
def get_ams_final_results(times):
    results = []
    for i in range(times):
        ams_median_estimate = run_ams_final_median(i+1,i+1)
        ams_mean_estimate = run_ams_final_mean(i+1,i+1)
        results.append((ams_median_estimate, ams_mean_estimate))
        print((ams_median_estimate, ams_mean_estimate))
    return results

In [ ]:
results = get_ams_final_results(10)
results

In [ ]:
results

In [ ]:
df =  pd.DataFrame(data=np.array(results),columns=["ams_median_estimate", "ams_mean_estimate"])
df

In [ ]:
sns.lineplot(data=df)